In [1]:
import ROOT as R
import array
import numpy as np
from math import log10
import pandas as pd
import datetime
import time
import deode_TL
import os
import FTT_spectrum as FFT_script
import socket
gemroc=0
acq_time=1

Welcome to JupyROOT 6.16/00


In [2]:
%jsroot on
c = R.TCanvas("c")
R.gStyle.SetOptStat(1)


In [3]:
def acquire_data(gemroc_num,run_time):
    HOST_IP="192.168.1.200"
    HOST_PORT=58880 + gemroc_num
    BUFSIZE=32000
    dataSock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    dataSock.settimeout(3)
    dataSock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    dataSock.bind((HOST_IP,HOST_PORT))
    data_list_tmp=[]
    start_time=time.time()
    while time.time()-start_time<run_time:
        data=dataSock.recv(BUFSIZE)
#         data="a"
        data_list_tmp.append(data)
    with open ("raw_data/efine_{}.raw".format(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")), 'wb+') as savefile:
        for item in data_list_tmp:
            savefile.write(item)
            
            
decode_istance=deode_TL.reader("raw_data/")

def decode_data(filename,gemroc):
    out_name=("root_files/")+filename.split("raw_data/")[1].split(".")[0]+".root"
    decode_istance.write_root(filename,gemroc, 0, 0, out_name)
    
def write_txt(filename,gemroc):
    out_name=("root_files/")+filename.split("raw_data/")[1].split(".")[0]+".txt"
    decode_istance.write_txt(filename,gemroc, 0, 0, out_name)
    
def get_last_file(mypath):
    onlyfiles = [f for f in os.listdir(mypath) if (os.path.isfile(os.path.join(mypath, f)) and 'missing_frames' not in f and "efine" in f)]
    onlyfiles.sort()
    return (os.path.join(mypath,onlyfiles[-1]))

In [32]:
acquire_data(gemroc,acq_time)
decode_data(get_last_file("raw_data/"),gemroc)


Sub 0 G 0 done in 0.290167


In [33]:
f=R.TFile(get_last_file("root_files/"))
# f=R.TFile("root_files/2020_08_07_15_07_06.root")
c = R.TCanvas("c")

ht=R.TH1I("ht","ht",10000,0,16)
f.tree.Draw("timestamp>>ht","")
c.Update()
c.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [34]:
f.tree.Draw("efine>>h(1024,0,1024)","")
c.Draw()

In [35]:
ext_1,ext_2=ht.GetBinLowEdge(ht.FindFirstBinAbove(0)),ht.GetBinCenter(ht.FindLastBinAbove(0))
R.gStyle.SetOptStat(0)


In [36]:
h_trans_hz_0, h_spec_0, h_ori=FFT_script.extract_spectrum(f.tree,100000,ext_1,ext_2)

In [38]:
h_spec_0.Draw()
c.Draw()

In [39]:
h_trans_hz_0.Draw()
c.Draw()

In [ ]:
f.tree.Show(2)

In [ ]:
last_timestamp=0
for hit in range (0,f.tree.GetEntries()):
    f.tree.GetEntry(hit)
    timestamp=getattr(f.tree ,"timestamp")
    diff=timestamp-last_timestamp
    last_timestamp=timestamp
    if abs(diff)>1e-04:
        tcoarse=getattr(f.tree ,"tcoarse")
        last_frame=getattr(f.tree ,"last_frame")
        timestamp = getattr(f.tree,"timestamp")
        print("Diff :{:.5f},\ttcoarse {:5.2f},\tlast_frame {:.2f},\ttimestamp {:.2f}".format(diff,tcoarse,last_frame,timestamp))
        f.tree.GetEntry(hit-1)
        timestamp=getattr(f.tree ,"timestamp")
        tcoarse=getattr(f.tree ,"tcoarse")
        last_frame=getattr(f.tree ,"last_frame")
        timestamp = getattr(f.tree,"timestamp")
        print("Prec : Diff :{:.5f},\ttcoarse {:5.2f},\tlast_frame {:.2f},\ttimestamp {:.2f}".format(diff,tcoarse,last_frame,timestamp))


In [ ]:
spect = R.TSpectrum(5)
spect.Search(h_trans_hz_0)
# xs=spect.GetPositionX()
# ys=spect.GetPositionY()
# peaks=np.zeros((spect.GetNPeaks(),2))
# for i in range (0,spect.GetNPeaks()):
#     peaks[i]=(xs[i],ys[i])
# srted=peaks[np.argsort(peaks[:,1])]
# srted=srted[::-1]
# for i in range (0,spect.GetNPeaks()):
#         if (srted[i,0]<500):
#              print (f'Freq {srted[i,0]:.2f},  amplitude {srted[i,1]:.2f}')
spect.Draw()
c.Draw()

In [ ]:
xs=spect.GetPositionX()

In [ ]:
for elem in range(0,5):
    print (xs[elem])

In [ ]:
rate=ht.Integral()/(ext_2-ext_1)

In [ ]:
ht.Integral()

In [ ]:
rate

In [ ]:
f.tree.Draw("tiger:channel","","colz")
c.Draw()

In [ ]:
rate/64

In [ ]:
h_rate= R.TH1F("h_rate","h_rate",64,0,64)
f.tree.Draw("channel>>h_rate","","colz")


In [ ]:
h_rate.Scale(1/(ext_2-ext_1))

In [ ]:
h_rate.Sumw2(1)
h_rate.Draw("")
c.Draw()

In [ ]:
h_rate_proj=R.TH1F("h_rate_proj","h_rate_proj",1000,0,100000)
for i in range (0, 64):
    h_rate_proj.Fill(h_rate.GetBinContent(i))

In [ ]:
R.gStyle.SetOptStat(1111)

h_rate_proj.Draw()


In [ ]:
c.Draw()